Spacy for Entity Recognition

References :
- https://spacy.io/usage/visualizers
- https://explosion.ai/blog/deep-learning-formula-nlp
- https://spacy.io/usage/training#ner

- video 
https://www.youtube.com/watch?v=l4scwf8KeIA
- training 
https://towardsdatascience.com/a-review-of-named-entity-recognition-ner-using-automatic-summarization-of-resumes-5248a75de175

## Creating training dataset

- format
    - ("Uber blew through $1 million a week", {'entities':[(0, 4, 'ORG')]}), 

### retrieve sample table data

In [ ]:
import db_conn
import pandas as pd

In [ ]:
conn = db_conn.get_connection()
cur = conn.cursor()

In [ ]:
cur.execute("""
select lower(table_title) from article_tables;
""")
titles = cur.fetchall()

In [ ]:
titles = list(map(lambda x: x[0], titles))
len(titles)

In [ ]:
titles[:5]

## retrieve all drug names

In [ ]:
cur.execute("""
SELECT distinct(lower(cui1_str)) FROM dict_collapsed_final;
""")
d1 = cur.fetchall()
d1 = pd.DataFrame(d1)
d1.head()

In [ ]:
cur.execute("""
SELECT distinct(lower(cui2_str)) FROM dict_collapsed_final;
""")
d2 = cur.fetchall()
d2 = pd.DataFrame(d2)
d2.head()

In [ ]:
drugs = d1.append(d2)
drugs.columns = ['name']
drugs.drop_duplicates(inplace=True)
drugs.reset_index(drop=True)

In [ ]:
drugs.to_csv('./drug_dictionary.csv', sep=',', header=None)

## retrieve all Side effects

In [ ]:
cur.execute("""
SELECT distinct(lower(llt_name)) from meddra_llt_181022;
""")
sd = pd.DataFrame(cur.fetchall())
sd.head()

In [ ]:
sd.columns = ['name']
sd.drop_duplicates(inplace=True)
sd.reset_index(drop=True)

## find drug names

In [ ]:
from flashtext import KeywordProcessor

In [ ]:
drug_keyword_processor = KeywordProcessor(case_sensitive=False)
for i, r in drugs.iterrows():
    drug_keyword_processor.add_keyword(r['name'].strip())

In [ ]:
ad_keyword_processor = KeywordProcessor(case_sensitive=False)
for i, sd in sd.iterrows():
    ad_keyword_processor.add_keyword(sd['name'].strip())

In [ ]:
c_entities = []
no_entities = []
for ti in titles:
    found = drug_keyword_processor.extract_keywords(ti.strip())
    ad_found = ad_keyword_processor.extract_keywords(ti.strip())
    ents = [] 
    for f in found:
        if ti.find(f)>=0:
            ents.append((ti.find(f), ti.find(f)+len(f), 'DRUG'))
        
    for f in ad_found:
        if ti.find(f)>=0:
            ents.append((ti.find(f), ti.find(f)+len(f), 'SIDE_EFFECT'))
        
    if len(ents)>0:
        c_entities.append((ti, {'entities':ents}))
    else:
        no_entities.append((ti, {'entities':[]}))

In [ ]:
len(c_entities)

In [ ]:
len(no_entities)

In [ ]:
c_entities[:5]

In [ ]:
com_list = c_entities + no_entities[:len(c_entities)]

Reference : https://spacy.io/usage/training#train-entity  

- prediction is based on the examples the model has seen during training.
- not memorizing -> why the training data should be representative of the data we want to process

In [ ]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [ ]:
import spacy
import random
nlp = spacy.load('en')

In [ ]:
# nlp = spacy.load(model)  # load existing spaCy model
# print("Loaded model '%s'" % model)

In [ ]:
# Add entity recognizer to model if it's not in the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
# otherwise, get it, so we can add labels to it
else:
    ner = nlp.get_pipe('ner')

In [ ]:
ner.add_label('DRUG') 
ner.add_label('SIDE_EFFECT')
# add new entity label to entity recognizer

In [ ]:
optimizer = nlp.entity.create_optimizer()

In [ ]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [ ]:
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(10):
        random.shuffle(com_list)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(com_list, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.1,#0.35,
                       losses=losses)
        print('Losses', losses)

In [ ]:
# test the trained model
no_entities[3500]

In [ ]:
# test_text = 'comparisons of responses to graded dobutamine infusion, y versus o'
test_text = 'Cosentyx, trade name Secukinumab, is a human antibody that binds to the protein interleukin (IL)-17A, and is marketed by Novartis for the treatment of psoriasis, ankylosing spondylitis, and psoriatic arthritis.'
# test_text = no_entities[3500][0]
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

In [ ]:
# save model to output directory

# output_dir = Path('./')
# if not output_dir.exists():
#     output_dir.mkdir()
# nlp.meta['name'] = new_model_name  # rename model
# nlp.to_disk(output_dir)
# print("Saved model to", output_dir)

# # test the saved model
# print("Loading from", output_dir)
# nlp2 = spacy.load(output_dir)
# doc2 = nlp2(test_text)
# for ent in doc2.ents:
#     print(ent.label_, ent.text)